# NHGIS County-Level Demographic Data Acquisition
## Swing State Election Analysis — CSCI 5502 Data Mining

**Purpose:** Programmatically acquire county-level demographic and socio-economic data from IPUMS NHGIS via their API for three swing states (Pennsylvania, Michigan, North Carolina) aligned with Presidential election years (2016, 2020, 2024).

**Data Source:** American Community Survey (ACS) 5-Year Estimates via [IPUMS NHGIS](https://www.nhgis.org/)  
**API Docs:** [IPUMS Developer Portal](https://developer.ipums.org/docs/v2/apiprogram/apis/nhgis/)  
**Python Library:** [`ipumspy`](https://ipumspy.readthedocs.io/en/latest/)

---

### How NHGIS API Works (Key Concepts)

NHGIS does **not** provide a live query API. Instead, it uses an **extract system**:

1. **Define** an extract request (which datasets, tables, geographic levels, extents)
2. **Submit** the request to IPUMS servers for background processing
3. **Wait** for processing to complete (typically 30 seconds–5 minutes)
4. **Download** the resulting ZIP file containing CSV data + codebook
5. **Read** the CSV into pandas for analysis

### Dataset Naming Convention

NHGIS organizes ACS data into datasets named by year range and variant:

| Election Year | ACS 5-Year Period | NHGIS Dataset Name | Coverage |
|---|---|---|---|
| **2016** | 2012–2016 | `2012_2016_ACS5a` | Block Groups & Larger (incl. County) |
| **2020** | 2016–2020 | `2016_2020_ACS5a` | Block Groups & Larger (incl. County) |
| **2024** | 2020–2024 | `2020_2024_ACS5a` | Block Groups & Larger (incl. County) |

The `a` suffix means the dataset includes tables available at the block group level and above.  
The `b` suffix includes additional detailed tables available only at tract level and above.  
We use `a` for most tables; we add `b` variants for specific detailed tables not in `a`.

### Geographic Extent Codes (State FIPS + "0")

| State | FIPS Code | NHGIS Extent Code |
|---|---|---|
| Pennsylvania | 42 | `420` |
| Michigan | 26 | `260` |
| North Carolina | 37 | `370` |

### ACS Data Tables Selected

These are the standard Census Bureau table codes, used identically in NHGIS:

| Table Code | Description | Key Variables |
|---|---|---|
| **B01003** | Total Population | Total pop count |
| **B01002** | Median Age by Sex | Median age |
| **B02001** | Race | White, Black, Asian, etc. |
| **B03002** | Hispanic/Latino Origin by Race | Hispanic breakdown |
| **B15003** | Educational Attainment (25+) | HS, Bachelor's, Graduate |
| **B19013** | Median Household Income | Dollar amount |
| **B19001** | Household Income (brackets) | Income distribution |
| **B17001** | Poverty Status | Below poverty line |
| **B23025** | Employment Status | Unemployed, labor force |
| **B25077** | Median Home Value | Dollar amount |
| **B25003** | Tenure (Own vs. Rent) | Owner-occupied, Renter |
| **B25064** | Median Gross Rent | Dollar amount |
| **B08301** | Means of Transportation to Work | Car, transit, WFH |
| **B11001** | Household Type | Family, nonfamily |

> **Note on B-table availability:** Most B-tables are in `ACS5a`. Some detailed tables (B15003 has 25 categories) are only in `ACS5b`. We'll verify availability via metadata queries.

---

## 0. Setup & Installation

In [ ]:
# Install dependencies (run once)
# !pip install ipumspy pandas python-dotenv

In [2]:
import os
import time
import json
from pathlib import Path
from zipfile import ZipFile

import pandas as pd
from dotenv import load_dotenv
from ipumspy import (
    IpumsApiClient,
    AggregateDataExtract,
    NhgisDataset,
    NhgisDatasetMetadata,
    NhgisDataTableMetadata,
    save_extract_as_json,
    define_extract_from_json,
)

# Load .env from repo root (walks up from notebook location)
load_dotenv()

print("Imports successful.")

Imports successful.


In [3]:
# ============================================================
# API KEY SETUP
# ============================================================
# 
# Setup (one-time):
#   1. Register at https://www.nhgis.org/
#   2. Get API key at https://account.ipums.org/api_keys
#   3. Create .env file in repo root: IPUMS_API_KEY=your_key
#   4. Make sure .env is in .gitignore (it should be)
# ============================================================

IPUMS_API_KEY = os.environ.get("IPUMS_API_KEY")

if not IPUMS_API_KEY:
    raise ValueError(
        "IPUMS_API_KEY not found!\n"
        "Create a .env file in the repo root with:\n"
        "IPUMS_API_KEY=your_key_here\n\n"
        "See API_KEY_SETUP_GUIDE.md for full instructions."
    )

ipums = IpumsApiClient(IPUMS_API_KEY)
print(f"API client initialized (key: {IPUMS_API_KEY[:8]}...)")

API client initialized (key: 59cba10d...)


In [4]:
# ============================================================
# PROJECT CONSTANTS
# ============================================================

# Download directory for extract files
DOWNLOAD_DIR = Path("data/nhgis_raw")
DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

# Output directory for processed data
OUTPUT_DIR = Path("data/nhgis_processed")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Geographic extent codes (state FIPS + "0")
STATE_EXTENTS = ["260", "370", "420"]  # MI, NC, PA

STATE_FIPS = {
    "26": "Michigan",
    "37": "North Carolina",
    "42": "Pennsylvania",
}

# ACS 5-Year datasets aligned with election years
ELECTION_DATASETS = {
    2016: "2012_2016_ACS5a",
    2020: "2016_2020_ACS5a",
    2024: "2020_2024_ACS5a",
}

# Backup: some detailed tables may only be in 'b' variants
ELECTION_DATASETS_B = {
    2016: "2012_2016_ACS5b",
    2020: "2016_2020_ACS5b",
    2024: "2020_2024_ACS5b",
}

# Core ACS data tables we want
CORE_TABLES = [
    "B01003",  # Total Population
    "B01002",  # Median Age by Sex
    "B02001",  # Race
    "B03002",  # Hispanic/Latino Origin by Race
    "B15003",  # Educational Attainment (25+)
    "B19013",  # Median Household Income
    "B19001",  # Household Income (brackets)
    "B17001",  # Poverty Status
    "B23025",  # Employment Status
    "B25077",  # Median Home Value
    "B25003",  # Tenure (Own vs. Rent)
    "B25064",  # Median Gross Rent
    "B08301",  # Means of Transportation to Work
    "B11001",  # Household Type
]

print(f"Will request {len(CORE_TABLES)} tables × {len(ELECTION_DATASETS)} election years")
print(f"States: {list(STATE_FIPS.values())}")
print(f"Download directory: {DOWNLOAD_DIR.absolute()}")

Will request 14 tables × 3 election years
States: ['Michigan', 'North Carolina', 'Pennsylvania']
Download directory: c:\Users\Antoni\OneDrive\Pulpit\CuBoulder\2 sem\Data Mining CSCI 5502\project\Data-Mining-Project-Group-1\notebooks\data\nhgis_raw


---
## 1. Explore Metadata — Verify Table Availability

Before submitting extracts, we verify that our target tables exist in each dataset and that county-level data is available.

In [5]:
def check_dataset_metadata(dataset_name):
    """
    Retrieve metadata for a single NHGIS dataset.
    Returns the metadata object with data_tables, geog_levels, geographic_instances.
    """
    ds_meta = NhgisDatasetMetadata(dataset_name)
    ipums.get_metadata(ds_meta)
    return ds_meta


# Check the 2020-2024 dataset as our reference
print("Checking 2020_2024_ACS5a metadata...")
ds_2024 = check_dataset_metadata("2020_2024_ACS5a")

print(f"\nDescription: {ds_2024.description}")
print(f"Number of data tables: {len(ds_2024.data_tables)}")
print(f"\nGeographic levels available:")
for level in ds_2024.geog_levels:
    print(f"  - {level['name']}: {level['description']} (extent selection: {level['hasGeogExtentSelection']})")

Checking 2020_2024_ACS5a metadata...

Description: 5-Year Data [2020-2024, Block Groups & Larger Areas]
Number of data tables: 416

Geographic levels available:
  - nation: Nation (extent selection: False)
  - region: Region (extent selection: False)
  - division: Division (extent selection: False)
  - state: State (extent selection: True)
  - state_260: American Indian Area/Alaska Native Area/Hawaiian Home Land--State (extent selection: True)
  - state_290: American Indian Area/Alaska Native Area/Hawaiian Home Land--Tribal Subdivision/Remainder--State (extent selection: True)
  - state_311: Metropolitan Statistical Area/Micropolitan Statistical Area--State (extent selection: True)
  - state_315: Metropolitan Statistical Area/Micropolitan Statistical Area--Metropolitan Division--State (extent selection: True)
  - state_331: Combined Statistical Area--State (extent selection: True)
  - state_333: Combined Statistical Area--Metropolitan Statistical Area/Micropolitan Statistical Area--Sta

In [6]:
def verify_tables_in_dataset(dataset_name, target_tables):
    """
    Check which of our target tables are available in a given dataset.
    Returns (found, missing) tuple of lists.
    """
    ds_meta = check_dataset_metadata(dataset_name)
    available = {t["name"] for t in ds_meta.data_tables}
    found = [t for t in target_tables if t in available]
    missing = [t for t in target_tables if t not in available]
    return found, missing


# Verify all tables across all election-year datasets
print("=" * 60)
print("TABLE AVAILABILITY CHECK")
print("=" * 60)

tables_by_dataset = {}

for year, ds_name in ELECTION_DATASETS.items():
    print(f"\n--- {year} election → {ds_name} ---")
    found, missing = verify_tables_in_dataset(ds_name, CORE_TABLES)
    tables_by_dataset[year] = {"dataset": ds_name, "found": found, "missing": missing}
    print(f"  Found: {len(found)}/{len(CORE_TABLES)} tables")
    if missing:
        print(f"  ⚠ Missing from 'a' variant: {missing}")
        # Check 'b' variant for missing tables
        ds_b = ELECTION_DATASETS_B[year]
        found_b, still_missing = verify_tables_in_dataset(ds_b, missing)
        if found_b:
            print(f"  ✓ Found in '{ds_b}': {found_b}")
            tables_by_dataset[year]["found_in_b"] = found_b
            tables_by_dataset[year]["b_dataset"] = ds_b
        if still_missing:
            print(f"  ✗ Not found in either variant: {still_missing}")
    else:
        print(f"  ✓ All tables available!")

TABLE AVAILABILITY CHECK

--- 2016 election → 2012_2016_ACS5a ---
  Found: 13/14 tables
  ⚠ Missing from 'a' variant: ['B17001']
  ✓ Found in '2012_2016_ACS5b': ['B17001']

--- 2020 election → 2016_2020_ACS5a ---
  Found: 13/14 tables
  ⚠ Missing from 'a' variant: ['B17001']
  ✓ Found in '2016_2020_ACS5b': ['B17001']

--- 2024 election → 2020_2024_ACS5a ---
  Found: 13/14 tables
  ⚠ Missing from 'a' variant: ['B17001']
  ✓ Found in '2020_2024_ACS5b': ['B17001']


In [7]:
# Examine a specific data table's variables (example: B02001 - Race)
print("Inspecting B02001 (Race) from 2020_2024_ACS5a:")
print("=" * 50)

table_meta = NhgisDataTableMetadata(name="B02001", dataset_name="2020_2024_ACS5a")
ipums.get_metadata(table_meta)

print(f"Table: {table_meta.description}")
print(f"Universe: {table_meta.universe}")
print(f"NHGIS Code: {table_meta.nhgis_code}")
print(f"\nVariables ({len(table_meta.variables)}):")
for var in table_meta.variables:
    print(f"  {var['nhgisCode']}: {var['description']}")

Inspecting B02001 (Race) from 2020_2024_ACS5a:
Table: Race
Universe: Total population
NHGIS Code: AUO7

Variables (10):
  AUO7001: Total
  AUO7002: White alone
  AUO7003: Black or African American alone
  AUO7004: American Indian and Alaska Native alone
  AUO7005: Asian alone
  AUO7006: Native Hawaiian and Other Pacific Islander alone
  AUO7007: Some Other Race alone
  AUO7008: Two or More Races
  AUO7009: Two or More Races: Two races including Some Other Race
  AUO7010: Two or More Races: Two races excluding Some Other Race, and three or more races


In [8]:
# Quick inspection of a few more key tables
for tbl_name in ["B19013", "B15003", "B17001"]:
    print(f"\n{'='*50}")
    print(f"Table: {tbl_name}")
    # Try in ACS5a first
    try:
        tbl = NhgisDataTableMetadata(name=tbl_name, dataset_name="2020_2024_ACS5a")
        ipums.get_metadata(tbl)
        print(f"  Found in ACS5a")
        print(f"  Description: {tbl.description}")
        print(f"  Universe: {tbl.universe}")
        print(f"  Variables: {len(tbl.variables)}")
    except Exception as e:
        print(f"  Not in ACS5a, checking ACS5b...")
        try:
            tbl = NhgisDataTableMetadata(name=tbl_name, dataset_name="2020_2024_ACS5b")
            ipums.get_metadata(tbl)
            print(f"  Found in ACS5b")
            print(f"  Description: {tbl.description}")
            print(f"  Variables: {len(tbl.variables)}")
        except Exception as e2:
            print(f"  ERROR: {e2}")


Table: B19013
  Found in ACS5a
  Description: Median Household Income in the Past 12 Months (in 2024 Inflation-Adjusted Dollars)
  Universe: Households
  Variables: 1

Table: B15003
  Found in ACS5a
  Description: Educational Attainment for the Population 25 Years and Over
  Universe: Population 25 years and over
  Variables: 25

Table: B17001
  Not in ACS5a, checking ACS5b...
  Found in ACS5b
  Description: Poverty Status in the Past 12 Months by Sex by Age
  Variables: 59


---
## 2. Define & Submit Extract Requests

We create one extract per election year. Each extract pulls all target tables at the county level, restricted to PA, MI, and NC.

In [9]:
def build_extract_for_year(election_year, tables_info):
    """
    Build an AggregateDataExtract for a single election year.
    Handles the case where some tables are in 'a' and some in 'b' variants.
    """
    info = tables_info[election_year]
    datasets = []

    # Primary dataset ('a' variant)
    if info["found"]:
        datasets.append(
            NhgisDataset(
                name=info["dataset"],
                data_tables=info["found"],
                geog_levels=["county"],
            )
        )

    # Secondary dataset ('b' variant) for tables not in 'a'
    if info.get("found_in_b"):
        datasets.append(
            NhgisDataset(
                name=info["b_dataset"],
                data_tables=info["found_in_b"],
                geog_levels=["county"],
            )
        )

    extract = AggregateDataExtract(
        collection="nhgis",
        description=f"Swing State Demographics - {election_year} Election (ACS 5yr)",
        datasets=datasets,
        geographic_extents=STATE_EXTENTS,  # MI, NC, PA only
    )

    return extract


# Build extracts for all three election years
extracts = {}
for year in ELECTION_DATASETS:
    extracts[year] = build_extract_for_year(year, tables_by_dataset)
    print(f"Built extract for {year}: {extracts[year]}")

Built extract for 2016: <ipumspy.api.extract.AggregateDataExtract object at 0x0000024051ABE660>
Built extract for 2020: <ipumspy.api.extract.AggregateDataExtract object at 0x0000024051B70F50>
Built extract for 2024: <ipumspy.api.extract.AggregateDataExtract object at 0x0000024051B702D0>


In [ ]:
# Save extract definitions to JSON for reproducibility
# NOTE: Run this AFTER cell-15 (submit) — requires submitted extracts
EXTRACT_DEFS_DIR = Path("data/extract_definitions")
EXTRACT_DEFS_DIR.mkdir(parents=True, exist_ok=True)

for year, ext in submitted_extracts.items():
    filepath = EXTRACT_DEFS_DIR / f"nhgis_extract_{year}.json"
    save_extract_as_json(ext, filepath)
    print(f"Saved extract definition: {filepath}")

print("\nExtract definitions saved. Share these with teammates for reproducibility.")

In [9]:
# ============================================================
# SUBMIT EXTRACTS TO IPUMS
# ============================================================
# This sends the requests to IPUMS servers for processing.
# Each extract typically processes in 30 seconds to 5 minutes.
# ============================================================

submitted_extracts = {}

for year, ext in extracts.items():
    print(f"Submitting extract for {year}...")
    ipums.submit_extract(ext)
    print(f"  → Extract #{ext.extract_id} submitted (status: queued)")
    submitted_extracts[year] = ext

print("\n✓ All extracts submitted. Waiting for processing...")

Submitting extract for 2016...
  → Extract #10 submitted (status: queued)
Submitting extract for 2020...
  → Extract #11 submitted (status: queued)
Submitting extract for 2024...
  → Extract #12 submitted (status: queued)

✓ All extracts submitted. Waiting for processing...


In [12]:
# ============================================================
# WAIT FOR COMPLETION & DOWNLOAD
# ============================================================

for year, ext in submitted_extracts.items():
    print(f"\nWaiting for {year} extract (#{ext.extract_id})...")
    ipums.wait_for_extract(ext, timeout=600)  # 10 min timeout
    print(f"  → Completed! Downloading...")

    ipums.download_extract(ext, download_dir=DOWNLOAD_DIR)
    print(f"  → Downloaded to {DOWNLOAD_DIR}")

print("\n" + "=" * 50)
print("✓ ALL EXTRACTS DOWNLOADED")
print("=" * 50)

# Show downloaded files
for f in sorted(DOWNLOAD_DIR.glob("*.zip")):
    print(f"  {f.name} ({f.stat().st_size / 1024:.1f} KB)")


Waiting for 2016 extract (#1)...
  → Completed! Downloading...
  → Downloaded to data\nhgis_raw

Waiting for 2020 extract (#2)...
  → Completed! Downloading...
  → Downloaded to data\nhgis_raw

Waiting for 2024 extract (#3)...
  → Completed! Downloading...
  → Downloaded to data\nhgis_raw

✓ ALL EXTRACTS DOWNLOADED
  nhgis0001_csv.zip (176.6 KB)
  nhgis0002_csv.zip (178.0 KB)
  nhgis0003_csv.zip (181.4 KB)


---
## 3. Load & Parse Downloaded Data

NHGIS delivers aggregate data as CSV files inside ZIP archives. Each dataset/table combination produces a separate CSV. We'll unpack them and load into pandas.

In [10]:
def load_nhgis_csvs_from_zip(zip_path):
    """
    Load all CSV files from an NHGIS extract ZIP.
    Returns a dict of {filename: DataFrame}.
    Skips codebook .txt files.
    """
    dfs = {}
    with ZipFile(zip_path) as z:
        csv_files = [n for n in z.namelist() if n.endswith(".csv")]
        print(f"  Found {len(csv_files)} CSV files in {zip_path.name}")
        for csv_name in csv_files:
            with z.open(csv_name) as f:
                df = pd.read_csv(f, encoding="latin-1", low_memory=False)
                dfs[csv_name] = df
                print(f"    {csv_name}: {df.shape[0]} rows × {df.shape[1]} cols")
    return dfs


# Load all downloaded extracts
all_data = {}
for zip_file in sorted(DOWNLOAD_DIR.glob("*.zip")):
    if "_csv" in zip_file.name:  # Data files (not shape files)
        print(f"\nLoading: {zip_file.name}")
        all_data[zip_file.name] = load_nhgis_csvs_from_zip(zip_file)


Loading: nhgis0001_csv.zip
  Found 2 CSV files in nhgis0001_csv.zip
    nhgis0001_csv/nhgis0001_ds226_20165_county.csv: 250 rows × 158 cols
    nhgis0001_csv/nhgis0001_ds225_20165_county.csv: 250 rows × 282 cols

Loading: nhgis0002_csv.zip
  Found 2 CSV files in nhgis0002_csv.zip
    nhgis0002_csv/nhgis0002_ds249_20205_county.csv: 250 rows × 282 cols
    nhgis0002_csv/nhgis0002_ds250_20205_county.csv: 250 rows × 158 cols

Loading: nhgis0003_csv.zip
  Found 2 CSV files in nhgis0003_csv.zip
    nhgis0003_csv/nhgis0003_ds272_20245_county.csv: 250 rows × 280 cols
    nhgis0003_csv/nhgis0003_ds273_20245_county.csv: 250 rows × 156 cols


In [11]:
# Examine the structure of one loaded file
# NHGIS CSVs contain identifying columns (GISJOIN, STATE, COUNTY, etc.) plus data columns

# Get the first dataframe from the first zip
if all_data:
    first_zip = list(all_data.keys())[0]
    first_csv = list(all_data[first_zip].keys())[0]
    sample_df = all_data[first_zip][first_csv]

    print(f"Sample file: {first_csv}")
    print(f"Shape: {sample_df.shape}")
    print(f"\nColumn names (first 30):")
    for col in sample_df.columns[:30]:
        print(f"  {col}")

    print(f"\nFirst 3 rows (identifying columns):")
    id_cols = [c for c in sample_df.columns if c in [
        "GISJOIN", "YEAR", "STATE", "STATEA", "COUNTY", "COUNTYA",
        "STUSAB", "NAME_E", "NAME_M"
    ]]
    display(sample_df[id_cols].head(3))
else:
    print("No data loaded yet. Run the extract/download cells above first.")

Sample file: nhgis0001_csv/nhgis0001_ds226_20165_county.csv
Shape: (250, 158)

Column names (first 30):
  GISJOIN
  YEAR
  STUSAB
  REGIONA
  DIVISIONA
  STATE
  STATEA
  COUNTY
  COUNTYA
  COUSUBA
  PLACEA
  TRACTA
  CONCITA
  AIANHHA
  RES_ONLYA
  TRUSTA
  AIHHTLI
  AITSCEA
  ANRCA
  CBSAA
  CSAA
  METDIVA
  NECTAA
  CNECTAA
  NECTADIVA
  UAA
  CDCURRA
  SLDUA
  SLDLA
  ZCTA5A

First 3 rows (identifying columns):


,GISJOIN,YEAR,STUSAB,STATE,STATEA,COUNTY,COUNTYA,NAME_E,NAME_M
0,G2600010,2012-2016,MI,Michigan,26,Alcona County,1,"Alcona County, Michigan","Alcona County, Michigan"
1,G2600030,2012-2016,MI,Michigan,26,Alger County,3,"Alger County, Michigan","Alger County, Michigan"
2,G2600050,2012-2016,MI,Michigan,26,Allegan County,5,"Allegan County, Michigan","Allegan County, Michigan"


---
## 4. Filter to Target States & Validate Coverage

Verify we have data for all counties in PA, MI, and NC.

In [12]:
# Expected county counts
EXPECTED_COUNTIES = {
    "Michigan": 83,
    "North Carolina": 100,
    "Pennsylvania": 67,
}


def validate_county_coverage(df, state_fips_col="STATEA"):
    """
    Check that we have the expected number of counties per state.
    STATEA column in NHGIS contains the state FIPS code (as string).
    """
    results = {}
    for fips, state_name in STATE_FIPS.items():
        state_data = df[df[state_fips_col].astype(str).str.strip() == fips]
        n_counties = state_data["COUNTYA"].nunique() if "COUNTYA" in df.columns else len(state_data)
        expected = EXPECTED_COUNTIES[state_name]
        status = "✓" if n_counties == expected else "⚠"
        results[state_name] = {"found": n_counties, "expected": expected, "status": status}
        print(f"  {status} {state_name}: {n_counties}/{expected} counties")
    return results


# Validate each loaded dataset
if all_data:
    for zip_name, csvs in all_data.items():
        print(f"\n{'='*50}")
        print(f"Validating: {zip_name}")
        for csv_name, df in csvs.items():
            print(f"\n  File: {csv_name}")
            if "STATEA" in df.columns:
                validate_county_coverage(df)
            else:
                print("  (No STATEA column — checking available ID columns)")
                print(f"  Columns: {[c for c in df.columns if not c.startswith(('A', 'B', 'C'))]}")


Validating: nhgis0001_csv.zip

  File: nhgis0001_csv/nhgis0001_ds226_20165_county.csv
  ✓ Michigan: 83/83 counties
  ✓ North Carolina: 100/100 counties
  ✓ Pennsylvania: 67/67 counties

  File: nhgis0001_csv/nhgis0001_ds225_20165_county.csv
  ✓ Michigan: 83/83 counties
  ✓ North Carolina: 100/100 counties
  ✓ Pennsylvania: 67/67 counties

Validating: nhgis0002_csv.zip

  File: nhgis0002_csv/nhgis0002_ds249_20205_county.csv
  ✓ Michigan: 83/83 counties
  ✓ North Carolina: 100/100 counties
  ✓ Pennsylvania: 67/67 counties

  File: nhgis0002_csv/nhgis0002_ds250_20205_county.csv
  ✓ Michigan: 83/83 counties
  ✓ North Carolina: 100/100 counties
  ✓ Pennsylvania: 67/67 counties

Validating: nhgis0003_csv.zip

  File: nhgis0003_csv/nhgis0003_ds272_20245_county.csv
  ✓ Michigan: 83/83 counties
  ✓ North Carolina: 100/100 counties
  ✓ Pennsylvania: 67/67 counties

  File: nhgis0003_csv/nhgis0003_ds273_20245_county.csv
  ✓ Michigan: 83/83 counties
  ✓ North Carolina: 100/100 counties
  ✓ Pennsy

---
## 5. Process & Structure the Data

Merge all tables for each election year into a single county-level DataFrame with meaningful column names.

In [13]:
# NHGIS identifying columns that we want to keep as our key
ID_COLUMNS = [
    "GISJOIN",    # Unique geographic identifier (stable across datasets)
    "YEAR",       # Data year
    "STUSAB",     # State abbreviation (e.g., PA, MI, NC)
    "STATE",      # State name
    "STATEA",     # State FIPS code
    "COUNTY",     # County name
    "COUNTYA",    # County FIPS code
]


def merge_nhgis_csvs(csv_dict):
    """
    Merge multiple NHGIS CSV DataFrames from a single extract
    on their shared GISJOIN key.
    """
    dfs = list(csv_dict.values())
    if not dfs:
        return pd.DataFrame()

    # Start with the first DataFrame
    merged = dfs[0].copy()

    # Merge subsequent DataFrames on GISJOIN
    for df in dfs[1:]:
        # Identify data columns (not ID columns)
        data_cols = [c for c in df.columns if c not in merged.columns]
        merge_cols = ["GISJOIN"] + data_cols
        merged = merged.merge(df[merge_cols], on="GISJOIN", how="outer")

    return merged


# Process each extract into a clean DataFrame
# (This section becomes active after data is downloaded)
if all_data:
    for zip_name, csvs in all_data.items():
        merged = merge_nhgis_csvs(csvs)
        print(f"Merged {zip_name}: {merged.shape}")
        # Show available ID columns
        avail_ids = [c for c in ID_COLUMNS if c in merged.columns]
        print(f"  ID columns: {avail_ids}")
        print(f"  Data columns: {merged.shape[1] - len(avail_ids)}")

Merged nhgis0001_csv.zip: (250, 400)
  ID columns: ['GISJOIN', 'YEAR', 'STUSAB', 'STATE', 'STATEA', 'COUNTY', 'COUNTYA']
  Data columns: 393
Merged nhgis0002_csv.zip: (250, 400)
  ID columns: ['GISJOIN', 'YEAR', 'STUSAB', 'STATE', 'STATEA', 'COUNTY', 'COUNTYA']
  Data columns: 393
Merged nhgis0003_csv.zip: (250, 398)
  ID columns: ['GISJOIN', 'YEAR', 'STUSAB', 'STATE', 'STATEA', 'COUNTY', 'COUNTYA']
  Data columns: 391


---
## 6. Build Unified County Demographics Panel

Create the final structured dataset: one row per county per election year, with key demographic variables derived from the raw NHGIS tables.



In [14]:
# Year-specific NHGIS code mappings 
# These were extracted from the codebook TXT files inside each ZIP.
# Each ACS edition uses DIFFERENT 4-letter prefixes for the same Census tables.

NHGIS_CODES = {
    2016: {  # nhgis0001 — ACS 2012-2016
        "B01003": "AF2L",   # Total Population
        "B01002": "AF2B",   # Median Age by Sex
        "B02001": "AF2M",   # Race
        "B03002": "AF2U",   # Hispanic/Latino Origin by Race
        "B08301": "AF3B",   # Means of Transportation to Work
        "B11001": "AF3J",   # Household Type
        "B15003": "AF4O",   # Educational Attainment (25+)
        "B17001": "AGI6",   # Poverty Status (from 'b' variant)
        "B19001": "AF48",   # Household Income Distribution
        "B19013": "AF49",   # Median Household Income
        "B23025": "AF67",   # Employment Status
        "B25003": "AF7P",   # Tenure (Owner/Renter)
        "B25064": "AF89",   # Median Gross Rent
        "B25077": "AF9L",   # Median Home Value
    },
    2020: {  # nhgis0002 — ACS 2016-2020
        "B01003": "AMPV",
        "B01002": "AMPL",
        "B02001": "AMPW",
        "B03002": "AMP3",
        "B08301": "AMQK",
        "B11001": "AMQS",
        "B15003": "AMRZ",
        "B17001": "AM63",   # from 'b' variant
        "B19001": "AMR7",
        "B19013": "AMR8",
        "B23025": "AMT9",
        "B25003": "AMUF",
        "B25064": "AMVZ",
        "B25077": "AMWB",
    },
    2024: {  # nhgis0003 — ACS 2020-2024
        "B01003": "AUO6",
        "B01002": "AUOW",
        "B02001": "AUO7",
        "B03002": "AUPF",
        "B08301": "AUPW",
        "B11001": "AUP1",
        "B15003": "AUQ8",
        "B17001": "AU77",   # from 'b' variant
        "B19001": "AURT",
        "B19013": "AURU",
        "B23025": "AUTW",
        "B25003": "AUUE",
        "B25064": "AUWG",
        "B25077": "AUWS",
    },
}

print("NHGIS code mappings loaded for 2016, 2020, 2024.")
print(f"  Example: B01003 (Total Population)")
for yr in [2016, 2020, 2024]:
    pfx = NHGIS_CODES[yr]["B01003"]
    print(f"    {yr}: column = {pfx}E001")


NHGIS code mappings loaded for 2016, 2020, 2024.
  Example: B01003 (Total Population)
    2016: column = AF2LE001
    2020: column = AMPVE001
    2024: column = AUO6E001


In [15]:
# Build derived variables for one year 

def col(codes, table, var_num):
    """Helper: build NHGIS column name like 'AF2LE001' from table code and variable number."""
    prefix = codes[table]
    return f"{prefix}E{var_num:03d}"


def build_demographics(df, election_year, codes):
    """
    Extract and compute ~30 derived demographic variables from a merged NHGIS DataFrame.
    
    Parameters:
        df: merged DataFrame (a + b variants joined on GISJOIN) for one election year
        election_year: 2016, 2020, or 2024
        codes: dict mapping Census table codes to NHGIS prefixes for this year
    
    Returns:
        DataFrame with one row per county, standardized column names
    """
    out = pd.DataFrame()
    
    # ── Identifiers ──
    out["county_fips"] = df["STATEA"].astype(str).str.zfill(2) + df["COUNTYA"].astype(str).str.zfill(3)
    out["gisjoin"] = df["GISJOIN"]
    out["state"] = df["STUSAB"]
    out["county_name"] = df["COUNTY"]
    out["election_year"] = election_year
    
    # ── B01003: Total Population ──
    out["total_population"] = df[col(codes, "B01003", 1)]
    
    # ── B01002: Median Age ──
    out["median_age"] = df[col(codes, "B01002", 1)]
    
    # ── B02001: Race (percentages) ──
    race_total = df[col(codes, "B02001", 1)]
    out["pct_white"] = df[col(codes, "B02001", 2)] / race_total * 100
    out["pct_black"] = df[col(codes, "B02001", 3)] / race_total * 100
    out["pct_asian"] = df[col(codes, "B02001", 5)] / race_total * 100
    out["pct_two_or_more_races"] = df[col(codes, "B02001", 8)] / race_total * 100
    
    # ── B03002: Hispanic/Latino Origin ──
    hisp_total = df[col(codes, "B03002", 1)]
    out["pct_hispanic"] = df[col(codes, "B03002", 12)] / hisp_total * 100
    out["pct_non_hispanic_white"] = df[col(codes, "B03002", 3)] / hisp_total * 100
    
    # ── B15003: Educational Attainment (25+) ──
    edu_total = df[col(codes, "B15003", 1)]
    # HS or higher = Regular HS diploma (17) + GED (18) + Some college (19,20) + Associate (21) + BA (22) + MA (23) + Professional (24) + Doctorate (25)
    hs_plus = sum(df[col(codes, "B15003", i)] for i in range(17, 26))
    out["pct_hs_or_higher"] = hs_plus / edu_total * 100
    # Bachelor's or higher = BA (22) + MA (23) + Professional (24) + Doctorate (25)
    ba_plus = sum(df[col(codes, "B15003", i)] for i in range(22, 26))
    out["pct_bachelors_plus"] = ba_plus / edu_total * 100
    # No HS diploma = vars 2-16 (no schooling through 12th grade no diploma)
    no_hs = sum(df[col(codes, "B15003", i)] for i in range(2, 17))
    out["pct_no_hs_diploma"] = no_hs / edu_total * 100
    
    # ── B17001: Poverty Status ──
    poverty_total = df[col(codes, "B17001", 1)]
    poverty_below = df[col(codes, "B17001", 2)]
    out["pct_below_poverty"] = poverty_below / poverty_total * 100
    
    # ── B19013: Median Household Income ──
    out["median_household_income"] = df[col(codes, "B19013", 1)]
    
    # ── B19001: Income Distribution (selected brackets) ──
    inc_total = df[col(codes, "B19001", 1)]
    # Low income: <$25k (vars 2-5)
    low_inc = sum(df[col(codes, "B19001", i)] for i in range(2, 6))
    out["pct_income_under_25k"] = low_inc / inc_total * 100
    # Middle income: $50k-$100k (vars 11-13)
    mid_inc = sum(df[col(codes, "B19001", i)] for i in range(11, 14))
    out["pct_income_50k_100k"] = mid_inc / inc_total * 100
    # High income: $100k+ (vars 14-17)
    high_inc = sum(df[col(codes, "B19001", i)] for i in range(14, 18))
    out["pct_income_over_100k"] = high_inc / inc_total * 100
    
    # ── B23025: Employment Status ──
    civilian_labor = df[col(codes, "B23025", 3)]
    unemployed = df[col(codes, "B23025", 5)]
    out["unemployment_rate"] = unemployed / civilian_labor * 100
    
    # ── B25003: Tenure (Owner vs Renter) ──
    tenure_total = df[col(codes, "B25003", 1)]
    out["pct_owner_occupied"] = df[col(codes, "B25003", 2)] / tenure_total * 100
    out["pct_renter_occupied"] = df[col(codes, "B25003", 3)] / tenure_total * 100
    
    # ── B25064: Median Gross Rent ──
    out["median_gross_rent"] = df[col(codes, "B25064", 1)]
    
    # ── B25077: Median Home Value ──
    out["median_home_value"] = df[col(codes, "B25077", 1)]
    
    # ── B08301: Means of Transportation to Work ──
    transport_total = df[col(codes, "B08301", 1)]
    out["pct_drive_alone"] = df[col(codes, "B08301", 3)] / transport_total * 100
    out["pct_carpool"] = df[col(codes, "B08301", 4)] / transport_total * 100
    out["pct_public_transit"] = df[col(codes, "B08301", 10)] / transport_total * 100
    out["pct_work_from_home"] = df[col(codes, "B08301", 21)] / transport_total * 100
    
    # ── B11001: Household Type ──
    hh_total = df[col(codes, "B11001", 1)]
    out["pct_family_households"] = df[col(codes, "B11001", 2)] / hh_total * 100
    out["pct_married_couple"] = df[col(codes, "B11001", 3)] / hh_total * 100
    out["pct_living_alone"] = df[col(codes, "B11001", 8)] / hh_total * 100
    
    return out


print("build_demographics() function defined.")
print("Derived variables (30):")
demo_vars = [
    "total_population", "median_age",
    "pct_white", "pct_black", "pct_asian", "pct_two_or_more_races",
    "pct_hispanic", "pct_non_hispanic_white",
    "pct_hs_or_higher", "pct_bachelors_plus", "pct_no_hs_diploma",
    "pct_below_poverty",
    "median_household_income", "pct_income_under_25k", "pct_income_50k_100k", "pct_income_over_100k",
    "unemployment_rate",
    "pct_owner_occupied", "pct_renter_occupied",
    "median_gross_rent", "median_home_value",
    "pct_drive_alone", "pct_carpool", "pct_public_transit", "pct_work_from_home",
    "pct_family_households", "pct_married_couple", "pct_living_alone",
]
for v in demo_vars:
    print(f"  • {v}")


build_demographics() function defined.
Derived variables (30):
  • total_population
  • median_age
  • pct_white
  • pct_black
  • pct_asian
  • pct_two_or_more_races
  • pct_hispanic
  • pct_non_hispanic_white
  • pct_hs_or_higher
  • pct_bachelors_plus
  • pct_no_hs_diploma
  • pct_below_poverty
  • median_household_income
  • pct_income_under_25k
  • pct_income_50k_100k
  • pct_income_over_100k
  • unemployment_rate
  • pct_owner_occupied
  • pct_renter_occupied
  • median_gross_rent
  • median_home_value
  • pct_drive_alone
  • pct_carpool
  • pct_public_transit
  • pct_work_from_home
  • pct_family_households
  • pct_married_couple
  • pct_living_alone


In [16]:
# Build panel from all 3 years 

# 'merged_data' should be the dict from Section 5: {zip_name: merged_DataFrame}
# Map zip names to election years


from pathlib import Path

# Re-merge from all_data (Section 5 didn't save to a dict)
def merge_csvs(csv_dict):
    dfs = list(csv_dict.values())
    merged = dfs[0].copy()
    for df in dfs[1:]:
        data_cols = [c for c in df.columns if c not in merged.columns]
        merge_cols = ["GISJOIN"] + data_cols
        merged = merged.merge(df[merge_cols], on="GISJOIN", how="outer")
    return merged

merged_data = {}
for zip_name, csvs in all_data.items():
    merged_data[zip_name] = merge_csvs(csvs)


ZIP_TO_YEAR = {
    "nhgis0001_csv.zip": 2016,
    "nhgis0002_csv.zip": 2020,
    "nhgis0003_csv.zip": 2024,
}

panels = []
for zip_name, year in ZIP_TO_YEAR.items():
    print(f"\nProcessing {year} ({zip_name})...")
    df = merged_data[zip_name]
    codes = NHGIS_CODES[year]
    
    demo = build_demographics(df, year, codes)
    panels.append(demo)
    print(f"  ✓ {len(demo)} counties, {len(demo.columns)} variables")

# Stack all years into one panel
panel = pd.concat(panels, ignore_index=True)
print(f"\n{'=' * 50}")
print(f"UNIFIED PANEL: {panel.shape[0]} rows × {panel.shape[1]} columns")
print(f"  Counties per year: {panel.groupby('election_year')['county_fips'].count().to_dict()}")
print(f"  States: {sorted(panel['state'].unique())}")
print(f"{'=' * 50}")

panel.head()


Processing 2016 (nhgis0001_csv.zip)...
  ✓ 250 counties, 33 variables

Processing 2020 (nhgis0002_csv.zip)...
  ✓ 250 counties, 33 variables

Processing 2024 (nhgis0003_csv.zip)...
  ✓ 250 counties, 33 variables

UNIFIED PANEL: 750 rows × 33 columns
  Counties per year: {2016: 250, 2020: 250, 2024: 250}
  States: ['MI', 'NC', 'PA']


,county_fips,gisjoin,state,county_name,election_year,total_population,median_age,pct_white,pct_black,pct_asian,...,pct_renter_occupied,median_gross_rent,median_home_value,pct_drive_alone,pct_carpool,pct_public_transit,pct_work_from_home,pct_family_households,pct_married_couple,pct_living_alone
0,26001,G2600010,MI,Alcona County,2016,10461,57.4,97.352070,0.248542,0.411051,...,11.582546,592,97500,82.127396,8.379716,0.216450,5.225727,62.718681,52.785039,33.259602
1,26003,G2600030,MI,Alger County,2016,9396,49.0,85.483184,7.322265,0.276713,...,13.574133,610,118500,72.304774,10.433133,1.802087,8.662662,64.608214,54.937373,31.255462
2,26005,G2600050,MI,Allegan County,2016,113666,40.0,94.399381,1.429627,0.653670,...,18.851700,757,144600,86.036905,7.493051,0.250926,3.972359,73.157232,60.170558,21.842412
3,26007,G2600070,MI,Alpena County,2016,28929,47.3,96.833627,0.594559,0.400982,...,23.109708,570,92700,82.642529,9.764089,0.614351,3.808978,62.991318,48.295185,31.681137
4,26009,G2600090,MI,Antrim County,2016,23215,50.0,95.899203,0.340297,0.267069,...,15.984148,699,145100,75.312067,13.389523,0.458765,7.841673,69.860787,58.449345,25.424246


In [17]:
# Quick sanity checks 

print("=== Sanity Checks ===\n")

# Check no missing county_fips
assert panel["county_fips"].notna().all(), "Missing county FIPS codes!"
print("✓ No missing county FIPS codes")

# Check expected row count
assert len(panel) == 750, f"Expected 750 rows, got {len(panel)}"
print("✓ 750 rows (250 counties × 3 years)")

# Check county counts per state per year
for yr in [2016, 2020, 2024]:
    subset = panel[panel["election_year"] == yr]
    mi = (subset["state"] == "MI").sum()
    nc = (subset["state"] == "NC").sum()
    pa = (subset["state"] == "PA").sum()
    assert mi == 83 and nc == 100 and pa == 67, f"County count mismatch in {yr}"
print("✓ County counts correct: MI=83, NC=100, PA=67 per year")

# Check percentage columns are in [0, 100]
pct_cols = [c for c in panel.columns if c.startswith("pct_")]
for c in pct_cols:
    vals = panel[c].dropna()
    if vals.min() < -0.01 or vals.max() > 100.01:
        print(f"  ⚠ {c}: range [{vals.min():.2f}, {vals.max():.2f}]")
    else:
        pass  # OK
print(f"✓ All {len(pct_cols)} percentage columns checked")

# Check population > 0
assert (panel["total_population"] > 0).all(), "Zero-population counties found!"
print("✓ All populations > 0")

# Summary stats
print(f"\n=== Summary Statistics ===")
panel.describe().round(2)

=== Sanity Checks ===

✓ No missing county FIPS codes
✓ 750 rows (250 counties × 3 years)
✓ County counts correct: MI=83, NC=100, PA=67 per year
✓ All 22 percentage columns checked
✓ All populations > 0

=== Summary Statistics ===


,election_year,total_population,median_age,pct_white,pct_black,pct_asian,pct_two_or_more_races,pct_hispanic,pct_non_hispanic_white,pct_hs_or_higher,...,pct_renter_occupied,median_gross_rent,median_home_value,pct_drive_alone,pct_carpool,pct_public_transit,pct_work_from_home,pct_family_households,pct_married_couple,pct_living_alone
count,750.00,750.00,750.00,750.00,750.00,750.00,750.00,750.00,750.00,750.00,...,750.00,750.00,750.00,750.00,750.00,750.00,750.00,750.00,750.00,750.00
mean,2020.00,132821.64,43.60,81.47,10.48,1.25,3.79,5.61,79.02,88.76,...,25.94,816.60,167238.00,79.33,9.42,0.73,6.64,65.19,49.48,29.19
std,3.27,234273.18,5.28,16.16,13.61,1.60,2.48,4.44,16.72,4.41,...,7.55,202.20,71395.81,5.05,2.18,1.75,4.01,4.37,5.55,3.66
min,2016.00,2102.00,26.40,24.17,0.00,0.00,0.02,0.10,23.25,72.05,...,8.38,431.00,68400.00,45.97,2.42,0.00,0.80,51.13,27.30,16.64
25%,2016.00,25611.25,40.60,73.86,1.16,0.42,1.98,2.17,69.95,86.35,...,20.91,677.25,116275.00,77.32,8.05,0.14,3.93,62.72,46.16,26.83
50%,2020.00,54794.00,43.30,88.24,3.94,0.63,3.12,4.35,85.25,89.55,...,25.27,769.00,152350.00,80.00,9.19,0.34,5.51,65.42,50.03,29.18
75%,2024.00,137647.25,46.78,93.52,14.26,1.32,4.94,7.53,92.26,91.78,...,30.14,906.75,197975.00,82.60,10.47,0.70,8.24,68.08,53.16,31.30
max,2024.00,1772259.00,60.00,99.22,62.04,9.23,15.29,27.98,97.68,97.12,...,48.55,1763.00,485600.00,90.15,25.40,25.71,29.06,81.37,65.56,40.58


In [18]:
# Add Population Density & Export to CSV 
# Drop if re-running
panel = panel.drop(columns=["land_area_sqmi", "population_density"], errors="ignore")
# Download county land area from Census Gazetteer (per-state files: MI=26, NC=37, PA=42)
import io, urllib.request

gaz_frames = []
for fips in ["26", "37", "42"]:
    url = f"https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2024_Gazetteer/2024_gaz_counties_{fips}.txt"
    response = urllib.request.urlopen(url)
    gaz_frames.append(pd.read_csv(io.BytesIO(response.read()), sep="\t", dtype={"GEOID": str}))

gaz = pd.concat(gaz_frames, ignore_index=True)
gaz = gaz[["GEOID", "ALAND_SQMI"]].rename(columns={"GEOID": "county_fips", "ALAND_SQMI": "land_area_sqmi"})

# Merge and compute density
panel = panel.merge(gaz, on="county_fips", how="left")
panel["population_density"] = panel["total_population"] / panel["land_area_sqmi"]

print(f"✓ Added land_area_sqmi and population_density")
print(f"  Missing area: {panel['land_area_sqmi'].isna().sum()}")
print(f"  Density range: {panel['population_density'].min():.1f} – {panel['population_density'].max():.1f} per sq mi")

# Export
OUTPUT_PATH = Path("data/county_demographics_panel.csv")
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

panel.to_csv(OUTPUT_PATH, index=False)
print(f"\n✓ Panel exported to {OUTPUT_PATH}")
print(f"  Shape: {panel.shape}")
print(f"  File size: {OUTPUT_PATH.stat().st_size / 1024:.1f} KB")

✓ Added land_area_sqmi and population_density
  Missing area: 0
  Density range: 3.9 – 11773.9 per sq mi

✓ Panel exported to data\county_demographics_panel.csv
  Shape: (750, 35)
  File size: 377.0 KB


---
## Appendix A: IPUMS NHGIS Citation

Required citation for any use of NHGIS data:

> Steven Manson, Jonathan Schroeder, David Van Riper, Katherine Knowles, Tracy Kugler, Finn Roberts, and Steven Ruggles. IPUMS National Historical Geographic Information System: Version 19.0 [dataset]. Minneapolis, MN: IPUMS. 2024. http://doi.org/10.18128/D050.V19.0

## Appendix B: Reproducibility

Extract definitions are saved as JSON files in `data/extract_definitions/`. Teammates can resubmit identical extracts by running:

```python
from ipumspy import define_extract_from_json, IpumsApiClient
ext = define_extract_from_json("data/extract_definitions/nhgis_extract_2024.json")
ipums = IpumsApiClient(os.environ["IPUMS_API_KEY"])
ipums.submit_extract(ext)
```

## Appendix C: Additional Table Ideas

If you want more differentiating variables, consider adding:

| Table | Description | Why Useful |
|---|---|---|
| B08006 | Sex of Workers by Transportation to Work | Commuting patterns |
| B16010 | Educational Attainment & Earnings | Income by education |
| B25071 | Median Gross Rent as % of Income | Housing affordability |
| B27001 | Health Insurance Coverage | Healthcare access |
| B14007 | School Enrollment by Level | Education engagement |
| B24011 | Occupation by Median Earnings | Economic structure |
| B99021 | Allocation of Household Type | Data quality flag |
| C24050 | Industry for Workers | Economic base |
| B05001 | Nativity & Citizenship | Immigration |
| B28002 | Internet Subscriptions | Digital access/rural proxy |